In [1]:
import os
import sys

# Get the current directory of the notebook
notebook_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory (one level up)
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to the system path
sys.path.append(parent_dir)
from utils import get_categorical_columns, get_numeric_columns, get_matchups

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 50)

### **Concatenating data**

In [3]:
dir_path = './raw_data/games'
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

In [4]:
dfs = []
for csv_file in csv_files:
    csv_path = os.path.join(dir_path, csv_file)
    df = pd.read_csv(csv_path)
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)

### Defining mutual cols

In [5]:
mutual_cols = ['SEASON_ID_y', 'GAME_DATE_y', 'MATCHUP_y']

rename_mutuals = {
    'SEASON_ID_x': 'SEASON_ID',
    'GAME_DATE_x': 'GAME_DATE',
    'MATCHUP_x': 'MATCHUP',
    'GAME_ID': 'GAME_ID'
}

keys = list(rename_mutuals.values())

df_raw = df_raw.drop(columns=mutual_cols, axis=1)
df_raw = df_raw.rename(columns=rename_mutuals)

In [6]:
df_raw.head(3)

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP,WL_x,MIN_x,PTS_x,FGM_x,FGA_x,FG_PCT_x,FG3M_x,FG3A_x,FG3_PCT_x,FTM_x,FTA_x,FT_PCT_x,OREB_x,DREB_x,REB_x,AST_x,STL_x,BLK_x,...,PLUS_MINUS_x,TEAM_ID_y,TEAM_ABBREVIATION_y,TEAM_NAME_y,WL_y,MIN_y,PTS_y,FGM_y,FGA_y,FG_PCT_y,FG3M_y,FG3A_y,FG3_PCT_y,FTM_y,FTA_y,FT_PCT_y,OREB_y,DREB_y,REB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PLUS_MINUS_y
0,22013,1610612737,ATL,Atlanta Hawks,21301166,2014-04-09,ATL vs. BOS,W,240,105,40,76,0.526,9,23,0.391,16,22,0.727,8,35,43,30,7,5,...,8.0,1610612738,BOS,Boston Celtics,L,240,97,37,84,0.440,12,26,0.462,11,14,0.786,11,28,39,25,7,3,14,22,-8.0
1,22013,1610612737,ATL,Atlanta Hawks,21300854,2014-02-26,ATL @ BOS,L,240,104,38,76,0.500,9,19,0.474,19,24,0.792,6,23,29,23,6,6,...,-11.0,1610612738,BOS,Boston Celtics,W,240,115,43,90,0.478,8,22,0.364,21,23,0.913,14,32,46,24,10,1,14,24,11.0
2,22013,1610612737,ATL,Atlanta Hawks,21300460,2013-12-31,ATL @ BOS,W,240,92,33,91,0.363,5,29,0.172,21,25,0.840,15,36,51,21,7,4,...,1.0,1610612738,BOS,Boston Celtics,L,239,91,35,84,0.417,5,20,0.250,16,17,0.941,8,39,47,28,6,7,15,22,-1.0


### Define home and away Team

In [7]:
cols = list(df_raw.columns)

cols_x = [col for col in cols if col.endswith('_x')]
cols_y = [col for col in cols if col.endswith('_y')]

cols_x.extend(keys)
cols_y.extend(keys)

df_x = df_raw[cols_x]
df_y = df_raw[cols_y]

df_away_x = df_x[df_x['MATCHUP'].str.contains('@')]
df_home_x = df_x[df_x['MATCHUP'].str.contains('vs')]

df_away_y = df_y[df_y['MATCHUP'].str.contains('vs')]
df_home_y = df_y[df_y['MATCHUP'].str.contains('@')]

In [8]:
rename_cols_away_x = {col: col.replace('_x', '_AWAY') for col in df_away_x.columns}
rename_cols_away_y = {col: col.replace('_y', '_AWAY') for col in df_away_y.columns}

rename_cols_home_x = {col: col.replace('_x', '_HOME') for col in df_home_x.columns}
rename_cols_home_y = {col: col.replace('_y', '_HOME') for col in df_home_y.columns}

df_away_x = df_away_x.rename(columns=rename_cols_away_x)
df_away_y = df_away_y.rename(columns=rename_cols_away_y)

df_home_x = df_home_x.rename(columns=rename_cols_home_x)
df_home_y = df_home_y.rename(columns=rename_cols_home_y)

In [9]:
df_home = pd.concat([df_home_x, df_home_y])
df_away = pd.concat([df_away_x, df_away_y])

In [10]:
all_games = pd.merge(df_home, df_away, on=keys)

In [11]:
all_games

,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,SEASON_ID,...,GAME_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY
0,1610612737,ATL,Atlanta Hawks,W,240,105,40,76,0.526,9,23,0.391,16,22,0.727,8,35,43,30,7,5,14,14,8.0,22013,...,21301166,1610612738,BOS,Boston Celtics,L,240,97,37,84,0.440,12,26,0.462,11,14,0.786,11,28,39,25,7,3,14,22,-8.0
1,1610612737,ATL,Atlanta Hawks,L,240,87,36,93,0.387,5,20,0.250,10,12,0.833,12,26,38,23,8,3,8,26,-7.0,22013,...,21300190,1610612738,BOS,Boston Celtics,W,240,94,33,72,0.458,4,16,0.250,24,31,0.774,7,40,47,18,3,7,17,14,7.0
2,1610612737,ATL,Atlanta Hawks,W,240,117,48,82,0.585,13,29,0.448,8,10,0.800,2,36,38,35,1,3,16,17,19.0,22013,...,21301132,1610612739,CLE,Cleveland Cavaliers,L,239,98,36,91,0.396,5,21,0.238,21,25,0.840,10,24,34,16,13,3,6,18,-19.0
3,1610612737,ATL,Atlanta Hawks,W,239,108,47,91,0.516,8,23,0.348,6,11,0.545,14,36,50,31,6,3,12,18,19.0,22013,...,21300282,1610612739,CLE,Cleveland Cavaliers,L,241,89,38,94,0.404,5,20,0.250,8,17,0.471,16,31,47,19,4,2,11,11,-19.0
4,1610612737,ATL,Atlanta Hawks,L,240,105,38,85,0.447,8,28,0.286,21,27,0.778,8,26,34,22,7,3,16,29,-6.0,22013,...,21301025,1610612740,NOP,New Orleans Pelicans,W,239,111,36,70,0.514,4,13,0.308,35,43,0.814,7,33,40,20,10,6,15,28,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12316,1610612752,NYK,New York Knicks,W,242,126,46,100,0.460,19,42,0.452,15,22,0.682,13,34,47,28,7,0,3,14,21.0,22023,...,22300161,1610612759,SAS,San Antonio Spurs,L,240,105,44,90,0.489,9,34,0.265,8,13,0.615,11,35,46,31,2,7,13,22,-21.0
12317,1610612754,IND,Indiana Pacers,W,240,152,55,95,0.579,20,38,0.526,22,23,0.957,9,40,49,38,7,4,14,24,41.0,22023,...,22300146,1610612759,SAS,San Antonio Spurs,L,242,111,40,94,0.426,8,26,0.308,23,30,0.767,12,24,36,26,7,5,11,15,-41.0
12318,1610612756,PHX,Phoenix Suns,L,241,121,43,80,0.538,16,32,0.500,19,26,0.731,5,29,34,26,5,4,14,15,-11.0,22023,...,22300131,1610612759,SAS,San Antonio Spurs,W,239,132,51,98,0.520,18,37,0.486,12,13,0.923,9,32,41,37,7,5,8,21,11.0
12319,1610612756,PHX,Phoenix Suns,L,239,114,42,83,0.506,18,39,0.462,12,15,0.800,6,37,43,29,8,7,19,18,-1.0,22023,...,22300113,1610612759,SAS,San Antonio Spurs,W,239,115,43,92,0.467,14,37,0.378,15,20,0.750,8,29,37,25,10,9,12,17,1.0


### Dropping columns

In [12]:
unuseful_cols = [
    'TEAM_ABBREVIATION_HOME',
    'TEAM_ABBREVIATION_AWAY'
]

In [13]:
all_games = all_games.drop(columns=unuseful_cols)

In [14]:
all_games

,TEAM_ID_HOME,TEAM_NAME_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,SEASON_ID,GAME_DATE,MATCHUP,GAME_ID,TEAM_ID_AWAY,TEAM_NAME_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY
0,1610612737,Atlanta Hawks,W,240,105,40,76,0.526,9,23,0.391,16,22,0.727,8,35,43,30,7,5,14,14,8.0,22013,2014-04-09,ATL vs. BOS,21301166,1610612738,Boston Celtics,L,240,97,37,84,0.440,12,26,0.462,11,14,0.786,11,28,39,25,7,3,14,22,-8.0
1,1610612737,Atlanta Hawks,L,240,87,36,93,0.387,5,20,0.250,10,12,0.833,12,26,38,23,8,3,8,26,-7.0,22013,2013-11-23,ATL vs. BOS,21300190,1610612738,Boston Celtics,W,240,94,33,72,0.458,4,16,0.250,24,31,0.774,7,40,47,18,3,7,17,14,7.0
2,1610612737,Atlanta Hawks,W,240,117,48,82,0.585,13,29,0.448,8,10,0.800,2,36,38,35,1,3,16,17,19.0,22013,2014-04-04,ATL vs. CLE,21301132,1610612739,Cleveland Cavaliers,L,239,98,36,91,0.396,5,21,0.238,21,25,0.840,10,24,34,16,13,3,6,18,-19.0
3,1610612737,Atlanta Hawks,W,239,108,47,91,0.516,8,23,0.348,6,11,0.545,14,36,50,31,6,3,12,18,19.0,22013,2013-12-06,ATL vs. CLE,21300282,1610612739,Cleveland Cavaliers,L,241,89,38,94,0.404,5,20,0.250,8,17,0.471,16,31,47,19,4,2,11,11,-19.0
4,1610612737,Atlanta Hawks,L,240,105,38,85,0.447,8,28,0.286,21,27,0.778,8,26,34,22,7,3,16,29,-6.0,22013,2014-03-21,ATL vs. NOP,21301025,1610612740,New Orleans Pelicans,W,239,111,36,70,0.514,4,13,0.308,35,43,0.814,7,33,40,20,10,6,15,28,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12316,1610612752,New York Knicks,W,242,126,46,100,0.460,19,42,0.452,15,22,0.682,13,34,47,28,7,0,3,14,21.0,22023,2023-11-08,SAS @ NYK,22300161,1610612759,San Antonio Spurs,L,240,105,44,90,0.489,9,34,0.265,8,13,0.615,11,35,46,31,2,7,13,22,-21.0
12317,1610612754,Indiana Pacers,W,240,152,55,95,0.579,20,38,0.526,22,23,0.957,9,40,49,38,7,4,14,24,41.0,22023,2023-11-06,SAS @ IND,22300146,1610612759,San Antonio Spurs,L,242,111,40,94,0.426,8,26,0.308,23,30,0.767,12,24,36,26,7,5,11,15,-41.0
12318,1610612756,Phoenix Suns,L,241,121,43,80,0.538,16,32,0.500,19,26,0.731,5,29,34,26,5,4,14,15,-11.0,22023,2023-11-02,SAS @ PHX,22300131,1610612759,San Antonio Spurs,W,239,132,51,98,0.520,18,37,0.486,12,13,0.923,9,32,41,37,7,5,8,21,11.0
12319,1610612756,Phoenix Suns,L,239,114,42,83,0.506,18,39,0.462,12,15,0.800,6,37,43,29,8,7,19,18,-1.0,22023,2023-10-31,SAS @ PHX,22300113,1610612759,San Antonio Spurs,W,239,115,43,92,0.467,14,37,0.378,15,20,0.750,8,29,37,25,10,9,12,17,1.0


In [16]:
all_games.columns

Index(['TEAM_ID_HOME', 'TEAM_NAME_HOME', 'WL_HOME', 'MIN_HOME', 'PTS_HOME',
       'FGM_HOME', 'FGA_HOME', 'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME',
       'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME', 'FT_PCT_HOME', 'OREB_HOME',
       'DREB_HOME', 'REB_HOME', 'AST_HOME', 'STL_HOME', 'BLK_HOME', 'TOV_HOME',
       'PF_HOME', 'PLUS_MINUS_HOME', 'SEASON_ID', 'GAME_DATE', 'MATCHUP',
       'GAME_ID', 'TEAM_ID_AWAY', 'TEAM_NAME_AWAY', 'WL_AWAY', 'MIN_AWAY',
       'PTS_AWAY', 'FGM_AWAY', 'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY',
       'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY', 'FT_PCT_AWAY',
       'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY', 'STL_AWAY',
       'BLK_AWAY', 'TOV_AWAY', 'PF_AWAY', 'PLUS_MINUS_AWAY'],
      dtype='object')

In [15]:
all_games.to_csv('./data/games.csv', index=False)

In [21]:
all_games[all_games['SEASON_ID'] == 22023].to_csv('training.csv')